In [13]:
import pandas;
import numpy as np;
import os.path;
import os;
import ast;
%matplotlib inline
np.set_printoptions(suppress=True)

In [14]:
def readCSVFile(file):
    data=pandas.read_csv(file,",",header=0, na_values='?', skipinitialspace=True);
    return data;
    pass;

def readTrainData(dataset):    
    return dataset.ix[:,6:], dataset.ix[:,4:5].astype(int),dataset.ix[:,5:6];
    pass;

def readTestData(dataset):    
    return dataset.ix[:,6:], dataset.ix[:,4:5].astype(int),dataset.ix[:,5:6];
    pass;

def normalizePhi(unNormalizedPhi,last_col_bias=False):    
    #assuming last column as bias column
    no_of_column=len(unNormalizedPhi[0]);
    phi=np.array(unNormalizedPhi);
    std=phi.std(0);
    mean=phi.mean(0);    
    std[no_of_column-1]=1;
    mean[no_of_column-1]=0;
    #phi_normalize=(phi-mean)/std;    
    
    max_vector=phi.max(axis=0)
    phi_normalize=phi/max_vector;    
    
    return phi_normalize;
    pass;

def writeTestData(test_id,ystar,filenumber=0,filename=None):
    if(filename==None):
        fo = open("log/output/sampleSubmission-"+str(filenumber)+".csv", "w");               
    else:
        fo = open(filename+".csv", "w");        
    fo.write("id,salary\n");
    m=len(ystar);
    for i in range(m):
        fo.write(str(test_id[i][0])+","+str(ystar[i][0])+"\n");
    fo.close();
    pass;

def dropColumns(dataframe,colList):
    for c in colList:
        dataframe.drop([c], axis = 1, inplace = True);
    pass;

def addColByCategory(dataset):
    return pandas.get_dummies(dataset);
    pass;

def categoryToNumber(dataset,categoryList):
    for c in categoryList:
        if (c in dataset):            
            dataset[c]=pandas.get_dummies(dataset[c]).values.argmax(1);        
    return dataset;
    pass;
    

def handleCategoryData(dataset,categoryList=None,byNumber=False):
    if(byNumber):
        return categoryToNumber(dataset,categoryList)
    else:
        return addColByCategory(dataset);
    
def findMostFrequentCount(dataset):
    #arr=dataset;
    #axis = 0
    #u, indices = np.unique(arr, return_inverse=True)
    #print(u);
    #u[np.argmax(np.apply_along_axis(np.bincount, axis, indices.reshape(arr.shape),None, np.max(indices) + 1), axis=axis)]
    x = np.array([0, 1, 1, 3, 2, 1, 2, 3]);
    w=np.bincount(dataset[:,1].astype(int))#work-class: Private
    o=np.bincount(dataset[:,6].astype(int))#occupation: Married-civ-spouse
    c=np.bincount(dataset[:,13].astype(int))#country : US
    pass;

def fillNanValue(dataframe,col,value):
    if (col in dataframe):
        dataframe[col].fillna(value, inplace=True);
    pass;

def imputeUnknowValue(dataframe):
    #by most frequent value;
    fillNanValue(dataframe,"workclass","Private");
    fillNanValue(dataframe,"occupation","Craft-repair");
    fillNanValue(dataframe,"native-country","United-States");
    pass;

def addRemainingCol(colList,dataframe,rowCount):
    i=0;
    for c in colList:
        if( c not in dataframe):
            dataframe.insert(i, c, 0);
        i+=1;
    pass;

In [151]:
class NN:
    logging_enabled=True;
    trained_ds=None;
    trained_output=None;
    no_of_hiddenlayer=0;
    neurons_per_hiddenlayer=0;    
    neurons_outlayer=10;
    weights=list();#list of matrix of weights
    layer_output=list();#vector of layers
    layer_delta=list(); 
    layer_error=list(); 
    no_of_iteration=1;
    mean_print_rate=10000;
    percent_print_rate=1000;
    learning_rate=1;
    approching_to_zero=1e-15;
    approching_to_one=1-1e-15;
    lamda=0.001;
    log_dir="log";
    log_folder=None;
    enable_bias_per_hidden=False;
    def __init__(self):
        np.random.seed(1);
        if(self.logging_enabled):
            self.log_folder=self.log_dir+"/"+self.getTimestamp();
            self.createDir(self.log_folder);            
        pass;
    
    def reInit(self):        
        self.layer_output=[0]*(self.no_of_hiddenlayer+2);
        self.layer_delta=[0]*(self.no_of_hiddenlayer+2);
        self.layer_error=[0]*(self.no_of_hiddenlayer+2);        
        pass;
        
    def createNN(self):
        self.no_of_features=len(self.trained_ds[0]);
        self.no_of_datapoint=len(self.trained_ds); 
        if(self.enable_bias_per_hidden and self.neurons_per_hiddenlayer>1):
            self.neurons_per_hiddenlayer+=1;#adding bias neuron whose o/p will always be 1;
            
        self.reInit();
        self.initWeightMatrix();       
        self.log("Neural Network Created...","-");
        pass;
    
    def initWeightMatrix(self):
        self.weights=list();
        for i in range(self.no_of_hiddenlayer):
            if(i==0):
                m=self.no_of_features;
            else:
                m=self.neurons_per_hiddenlayer;
            w_matrix = 2*np.random.random((m,self.neurons_per_hiddenlayer)) - 1;            
            
            if(self.enable_bias_per_hidden):
                #making weights of bias neuron as zero. So that its o/p is always one.
                w_matrix[:,0]=0;
                
            self.weights.append(w_matrix);
            #print(w_matrix);
            
        #last layer weight: For single output
        if(self.no_of_hiddenlayer==0):
            n=self.no_of_features;
        else:
            n=self.neurons_per_hiddenlayer;
        w_matrix=2*np.random.random((n,self.neurons_outlayer)) - 1;
        #print(w_vector);
        self.weights.append(w_matrix);
        self.weights=np.array(self.weights);
        pass;
        
    def getTimestamp(self):
        import os.path;
        import datetime;
        import time;
        ts = datetime.datetime.fromtimestamp(time.time()).strftime('%d-%m-%Y-%H:%M:%S')
        return ts;

    def createDir(self,directory):
        import os.path;
        if not os.path.exists(directory):
            os.makedirs(directory);
        pass;
    
    def writeWeights(self,filenumber=0):
        fname=self.log_folder+"/weights-"+str(filenumber)+"-";
        m=len(self.weights);
        for i in range(m):
            np.save(fname+""+str(i), nn.weights[i])        
        pass;
    
    def writeWeights(self,filenumber=0):
        fname=self.log_folder+"/weights-"+str(filenumber)+"-";
        m=len(self.weights);
        for i in range(m):
            np.save(fname+""+str(i), self.weights[i])        
        pass;
    import os;


    def writeFinalWeights(self):
        filename="weights";        
        np.save(filename, self.weights);
        os.rename(filename+".npy", filename+".txt");
        pass;

    
    def loadWeights(self,folder=None,filename_offset="weights-0-",filename=None):        
        w=list();
        i=0;
        if(filename==None):
            file_path=folder+"/"+filename_offset+str(i)+".npy";        
            while (os.path.isfile(file_path)):
                w.append(np.load(file_path));
                i+=1;
                file_path=folder+"/"+filename_offset+str(i)+".npy";
            self.weights=w;
        else:
            if(os.path.isfile(filename)):
                self.weights=np.load(filename);
                w=self.weights;
        return w;
    pass

    def activationFunction(self,x,deriv=False):
        return self.actFunSigmoid(x,deriv);
    
    def actFunSigmoid(self,x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))
    
    def actFunReLu(self,x,deriv=False):
        p = x > 0;            
        if(deriv==True):
            return p.astype(int);
        return x * p;
    
    def forwardPropogation(self,datapoints):        
        #intial layer i.e l0 zeroth layer
        self.layer_output[0]=datapoints;
        for i in range(1,self.no_of_hiddenlayer+2):
            prev_layer=self.layer_output[i-1];
            w=self.weights[i-1]
            sum_l=prev_layer.dot(w);           
            l_i=self.activationFunction(sum_l);
            self.layer_output[i]=l_i;   
            if(self.enable_bias_per_hidden and i<=self.no_of_hiddenlayer):
                #making weights of bias neuron as zero. So that its o/p is always one.
                l_i[:,0]=1;
            #print("l"+str(i)+":",l_i);
        return self.layer_output[self.no_of_hiddenlayer+1];
        pass;
    
    def findError(self):
        self.backPropogation(sel.layer_output);
        return self.getMeanError();
        
    def backPropogation(self,l_output):
        last_layer=len(l_output)-1;            
        
        #(target-output)        
        li=l_output[last_layer];
        error_diff=self.trained_output-li;
        delta=error_diff*self.activationFunction(li,deriv=True)    
        self.layer_delta[last_layer]=delta;
        self.layer_error[last_layer]=error_diff;
        
        for i in range(last_layer-1,0,-1):            
            #i-1 th layer calculation of delta
            error_diff=self.layer_delta[i+1].dot(self.weights[i].T);
            #print("l"+str(i)+"_error:",delta);
            delta=error_diff*self.activationFunction(l_output[i],deriv=True)
            self.layer_delta[i]=delta;
            self.layer_error[i]=error_diff;  
            
        #print("l"+str(i)+"_delta:",delta);        
        pass;
    
    def updateWeights(self,l_delta):
        #print(self.learning_rate);
        for i in range(self.no_of_hiddenlayer,-1,-1): # loop upto 0
            #print(str(i)+"==>a",self.weights[i][0]);            
            #print(str(i)+"==>b",self.layer_output[i].T.dot(l_delta[i+1])[0])
            #print(str(i)+"==>c",(self.lamda*self.weights[i])[0])            
            self.weights[i]+=self.learning_rate*self.layer_output[i].T.dot(l_delta[i+1]);
            if(self.lamda!=0):
                self.weights[i]-=self.learning_rate*-((self.lamda)*self.weights[i]);
            #print("w"+str(i)+":",self.layer_output[i].T.dot(l_delta[i+1]));
    
    
    def getMeanError(self):   
        return np.mean(np.abs(self.layer_error[self.no_of_hiddenlayer+1]),axis=0);
    
    def getTrainedStatus(self,predicted_y,actual_y):
        index = ["nn"];
        columns = ["Misclassified", "Accuracy"];
        status = pandas.DataFrame(index=index, columns=columns);    
        print("s:p",len(predicted_y));
        t_y=self.getThresholdValue(predicted_y);
        size=len(t_y);
        #error_diff=actual_y-predicted_y;
        #mean=np.mean(np.abs(error_diff),axis=0);
        misclassifed= np.array(list((1 if (actual_y[i] != t_y[i]).sum() != 0 else 0) for i in range(size))).sum();
        print(misclassifed)
        accuracy = float(((size - misclassifed)*100) / size);
        status.ix["nn"]=[misclassifed,accuracy];
        return status;
    
    def gradientDescent(self):
        for i in range(self.no_of_iteration):
            result=self.forwardPropogation(self.trained_ds);
            self.backPropogation(self.layer_output);            
            
            if (i%self.percent_print_rate==0 or i >=self.no_of_iteration-6):
                self.log("\n"+str(i)+") Percent completed: "+str((i*100)/self.no_of_iteration));
                self.writeWeights();     
            
            if ((i%self.mean_print_rate==0 or i >=self.no_of_iteration-6)):
                self.log("Error:"+str(self.getMeanError()));
                status=self.getTrainedStatus(result,self.trained_output);
                self.log("Misclassidied: ",status.ix["nn"]["Misclassified"]);
                self.log("Accuracy: ",status.ix["nn"]["Accuracy"]);
                
                
            if(i!=self.no_of_iteration-1):#donot update for last iteration            
                self.updateWeights(self.layer_delta);
        
        if self.logging_enabled:
            self.writeWeights();
            print("Error:",self.getMeanError());                
        pass;
    
    def getThresholdValue(self,result,threshold=0.5):
        return(result>=threshold).astype(int);

    def train(self):
         self.gradientDescent();
            
    def predict(self,datapoints):#forward propogation
        loutput=[0]*(self.no_of_hiddenlayer+2);
        #intial layer i.e l0 zeroth layer
        loutput[0]=datapoints;
        for i in range(1,self.no_of_hiddenlayer+2):
            prev_layer=loutput[i-1];
            w=self.weights[i-1]
            sum_l=prev_layer.dot(w);           
            l_i=self.activationFunction(sum_l);
            loutput[i]=l_i;   
            if(self.enable_bias_per_hidden and i<=self.no_of_hiddenlayer):
                #making weights of bias neuron as zero. So that its o/p is always one.
                l_i[:,0]=1;
        return loutput[self.no_of_hiddenlayer+1];
        pass;
    
    def log(self,text,data=None):
        if self.logging_enabled:
            if(data!=None):
                print(text,data);
            else:
                print(text);
        pass;
pass;

In [152]:
#--settings--
pandas.set_option('display.max_columns', None);
#---init---
dir="data/"
trainFile=dir+"train.csv";
testFile=dir+"test.csv";
drop_col=[]
trained_dataset=readCSVFile(trainFile);
trained_data,trained_y,trained_y_vector=readTrainData(trained_dataset);

test_dataset=readCSVFile(testFile);
test_data,test_y,test_y_vector=readTestData(test_dataset);

#droping unrelated-columns
#dropColumns(trained_data,drop_col);
#dropColumns(test_data,drop_col);

#converting panda data frame to numpy martix
mtx_train=trained_data.as_matrix(columns=None)
mtx_test=test_data.as_matrix(columns=None)
mtx_trained_y=trained_y.as_matrix(columns=None);
mtx_test_y=test_y.as_matrix(columns=None);
mtx_trained_y_vector=trained_y_vector.as_matrix(columns=None);
mtx_test_y_vector=test_y_vector.as_matrix(columns=None);
mtx_trained_y_vector=np.array(list((ast.literal_eval(e[0]) for e in mtx_trained_y_vector)));
mtx_test_y_vector=np.array(list((ast.literal_eval(e[0]) for e in mtx_test_y_vector)));

#adding bias column
mtx_train=np.column_stack((mtx_train,np.ones((len(mtx_train),1))))
mtx_test=np.column_stack((mtx_test,np.ones((len(mtx_test),1))))

#normalization
mtx_train_norm=normalizePhi(mtx_train)
mtx_test_norm=normalizePhi(mtx_test)


#print(mtx_train)
#print(mtx_train_norm)
#print(mtx_test_y_vector)
#pandas.get_dummies(trained_data.ix[:,1:2])
print("train",np.shape(mtx_train_norm),"test",np.shape(mtx_test_norm));

('train', (801, 31), 'test', (200, 31))


In [159]:
#neural network
#dataset: census income
nn= NN();
x=mtx_train_norm;
y=mtx_trained_y_vector;
nn.no_of_hiddenlayer=4;
nn.neurons_per_hiddenlayer=50;
nn.no_of_iteration=500000;
nn.learning_rate=0.001;
nn.lamda=0;
nn.mean_print_rate=1000;
nn.percent_print_rate=1000;
nn.enable_bias_per_hidden=True;

#Common setting
nn.logging_enabled=True;
nn.trained_ds=x;
nn.trained_output=y;
nn.createNN();
nn.loadWeights(filename="weights.txt");
nn.train();
nn.writeFinalWeights();
train_result=nn.predict(x);
print("---------------------------[Train]----------------------");
print("Predicted:",train_result);
print("");
print("Predicted-Th",nn.getThresholdValue(train_result));
print("");
print("Actual:",y);
status=nn.getTrainedStatus(train_result,mtx_trained_y_vector);
print("Misclassified: ",status.ix["nn"]["Misclassified"]);
print("Accuracy: ",status.ix["nn"]["Accuracy"]);

#test
print("---------------------------[Test]----------------------");
test_result=nn.predict(mtx_test_norm);
print("Pred Test:",test_result);
print("");
print("Predicted-Th",nn.getThresholdValue(test_result));
print("");
print("Act Test:",mtx_test_y_vector);
status=nn.getTrainedStatus(test_result,mtx_test_y_vector);
print("Misclassified: ",status.ix["nn"]["Misclassified"]);
print("Accuracy: ",status.ix["nn"]["Accuracy"]);

#writeTestData(mtx_test_id,result,filenumber=nn.getMeanError());
#writeTestData(mtx_test_id,nn.getThresholdValue(result),filename="predictions");

('Neural Network Created...', '-')

0) Percent completed: 0
Error:[ 0.00251377  0.00114947  0.00850517  0.0081748   0.00649942  0.00166652
  0.00685125  0.00155669  0.00683839  0.00456182]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

1000) Percent completed: 0
Error:[ 0.00248645  0.00113945  0.00848524  0.0081384   0.0064665   0.00165432
  0.00683085  0.00154233  0.00678001  0.00451964]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

2000) Percent completed: 0
Error:[ 0.00246078  0.0011297   0.00846632  0.00810376  0.00643543  0.00164238
  0.00681188  0.00152869  0.00672658  0.00448041]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

3000) Percent completed: 0
Error:[ 0.00243648  0.00112023  0.00844823  0.00807065  0.00640591  0.0016307
  0.00679405  0.00151564  0.00667728  0.00444367]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

4000) Percent completed: 0
Error:[ 0.00241334  0.001111    0.00843084  0.00803892  0.00637771  0


38000) Percent completed: 7
Error:[ 0.00190561  0.00089478  0.0080242   0.00735746  0.00577406  0.00133336
  0.00642519  0.00121265  0.00584404  0.00371487]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

39000) Percent completed: 7
Error:[ 0.00189514  0.00089027  0.00801543  0.00734392  0.00576208  0.00132709
  0.00641811  0.00120648  0.00583062  0.00370119]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

40000) Percent completed: 8
Error:[ 0.00188483  0.00088582  0.00800679  0.00733061  0.00575031  0.00132091
  0.00641113  0.0012004   0.00581747  0.00368773]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

41000) Percent completed: 8
Error:[ 0.00187469  0.00088144  0.00799826  0.00731752  0.00573873  0.00131481
  0.00640427  0.00119441  0.00580458  0.00367449]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

42000) Percent completed: 8
Error:[ 0.00186469  0.00087712  0.00798986  0.00730464  0.00572734  0.0013088
  0.0063975   0.0


76000) Percent completed: 15
Error:[ 0.00159526  0.0007602   0.00775872  0.00696197  0.005425    0.001143
  0.00621334  0.00102743  0.00546804  0.00331124]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

77000) Percent completed: 15
Error:[ 0.00158893  0.00075744  0.00775318  0.006954    0.00541799  0.00113902
  0.00620895  0.0010236   0.00546073  0.00330302]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

78000) Percent completed: 15
Error:[ 0.00158267  0.0007547   0.00774771  0.00694612  0.00541106  0.00113507
  0.00620461  0.0010198   0.00545351  0.00329489]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

79000) Percent completed: 15
Error:[ 0.00157647  0.00075199  0.00774229  0.00693834  0.00540421  0.00113117
  0.00620031  0.00101605  0.00544638  0.00328685]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

80000) Percent completed: 16
Error:[ 0.00157035  0.00074931  0.00773692  0.00693064  0.00539744  0.00112731
  0.00619606 


114000) Percent completed: 22
Error:[ 0.00139514  0.00067216  0.00758199  0.00671188  0.00520514  0.00101537
  0.0060731   0.00090522  0.00524167  0.00305147]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

115000) Percent completed: 23
Error:[ 0.00139078  0.00067023  0.00757811  0.00670647  0.00520039  0.00101255
  0.00607002  0.00090253  0.00523684  0.00304581]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

116000) Percent completed: 23
Error:[ 0.00138647  0.00066831  0.00757426  0.00670112  0.00519569  0.00100975
  0.00606695  0.00089987  0.00523205  0.00304021]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

117000) Percent completed: 23
Error:[ 0.00138219  0.00066641  0.00757044  0.00669581  0.00519103  0.00100698
  0.00606391  0.00089723  0.00522731  0.00303466]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

118000) Percent completed: 23
Error:[ 0.00137795  0.00066453  0.00756665  0.00669055  0.00518641  0.00100423
  0.00


152000) Percent completed: 30
Error:[ 0.00125244  0.00060851  0.00745396  0.00653548  0.00505025  0.00092205
  0.00597097  0.00081655  0.00508487  0.00286616]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

153000) Percent completed: 30
Error:[ 0.00124922  0.00060707  0.00745106  0.00653151  0.00504676  0.00091992
  0.00596864  0.00081453  0.00508136  0.00286197]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

154000) Percent completed: 30
Error:[ 0.00124602  0.00060563  0.00744817  0.00652758  0.00504331  0.00091781
  0.00596633  0.00081252  0.00507789  0.00285782]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

155000) Percent completed: 31
Error:[ 0.00124285  0.0006042   0.00744531  0.00652367  0.00503988  0.00091571
  0.00596404  0.00081053  0.00507443  0.0028537 ]
('s:p', 801)
20
('Misclassidied: ', 20)
('Accuracy: ', 97.0)

156000) Percent completed: 31
Error:[ 0.00123969  0.00060279  0.00744246  0.00651979  0.00503647  0.00091362
  0.00

...


342000) Percent completed: 68
Error:[ 0.00091762  0.00045026  0.00716271  0.00502941  0.00469697  0.00070606
  0.0056935   0.0005601   0.00481438  0.00244916]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

343000) Percent completed: 68
Error:[ 0.000915    0.00044974  0.00716023  0.00502339  0.0046946   0.0007046
  0.00569203  0.00055901  0.00481014  0.00244601]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

344000) Percent completed: 68
Error:[ 0.00091246  0.00044923  0.00715782  0.00501763  0.00469228  0.00070317
  0.0056906   0.00055796  0.00480604  0.00244294]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

345000) Percent completed: 69
Error:[ 0.00091001  0.00044871  0.00715547  0.00501209  0.00469     0.00070175
  0.00568919  0.00055695  0.00480206  0.00243993]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

346000) Percent completed: 69
Error:[ 0.00090762  0.00044818  0.00715319  0.00500677  0.00468777  0.00070036
  0.005


380000) Percent completed: 76
Error:[ 0.00084915  0.00042978  0.00709682  0.00488909  0.00462797  0.00066126
  0.00565169  0.00053074  0.00470304  0.00235962]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

381000) Percent completed: 76
Error:[ 0.0008478   0.00042925  0.00709552  0.00488661  0.00462651  0.00066028
  0.00565082  0.0005301   0.00470088  0.00235776]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

382000) Percent completed: 76
Error:[ 0.00084645  0.00042872  0.00709423  0.00488416  0.00462507  0.00065931
  0.00564996  0.00052946  0.00469875  0.00235591]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

383000) Percent completed: 76
Error:[ 0.00084512  0.0004282   0.00709296  0.00488175  0.00462363  0.00065834
  0.00564911  0.00052882  0.00469664  0.00235408]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

384000) Percent completed: 76
Error:[ 0.0008438   0.00042767  0.00709169  0.00487936  0.00462221  0.00065738
  0.00


418000) Percent completed: 83
Error:[ 0.00080395  0.00041076  0.00705427  0.0048111   0.00457882  0.00062799
  0.00562247  0.00050794  0.0046345   0.00229706]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

419000) Percent completed: 83
Error:[ 0.00080289  0.00041029  0.00705331  0.00480938  0.00457766  0.00062721
  0.00562179  0.00050737  0.00463299  0.00229559]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

420000) Percent completed: 84
Error:[ 0.00080184  0.00040982  0.00705235  0.00480767  0.0045765   0.00062643
  0.0056211   0.00050681  0.00463149  0.00229414]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

421000) Percent completed: 84
Error:[ 0.00080079  0.00040936  0.0070514   0.00480597  0.00457535  0.00062566
  0.00562042  0.00050625  0.00463001  0.00229268]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

422000) Percent completed: 84
Error:[ 0.00079974  0.0004089   0.00705046  0.00480428  0.00457421  0.00062489
  0.00


456000) Percent completed: 91
Error:[ 0.00076674  0.00039422  0.00702265  0.00475306  0.0045379   0.00060091
  0.00559843  0.00048736  0.00458446  0.00224587]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

457000) Percent completed: 91
Error:[ 0.00076583  0.00039382  0.00702197  0.00475171  0.00453689  0.00060027
  0.00559785  0.00048684  0.00458333  0.00224463]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

458000) Percent completed: 91
Error:[ 0.00076492  0.00039342  0.00702131  0.00475036  0.00453589  0.00059964
  0.00559727  0.00048631  0.00458221  0.0022434 ]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

459000) Percent completed: 91
Error:[ 0.00076402  0.00039303  0.00702065  0.00474903  0.00453489  0.00059901
  0.0055967   0.00048579  0.00458109  0.00224217]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

460000) Percent completed: 92
Error:[ 0.00076312  0.00039264  0.00702     0.0047477   0.0045339   0.00059838
  0.00


494000) Percent completed: 98
Error:[ 0.00073222  0.00038243  0.00701726  0.00470921  0.0045007   0.00058264
  0.00557907  0.00046717  0.0045522   0.00220173]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

495000) Percent completed: 99
Error:[ 0.00073104  0.00038251  0.00701974  0.00470864  0.00449969  0.00058285
  0.00557873  0.00046663  0.00455238  0.00220064]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

496000) Percent completed: 99
Error:[ 0.00072974  0.00038273  0.0070231   0.00470824  0.0044987   0.00058328
  0.00557844  0.00046612  0.00455287  0.00219956]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

497000) Percent completed: 99
Error:[ 0.00072827  0.00038314  0.00702785  0.00470811  0.00449773  0.00058407
  0.00557819  0.00046568  0.00455383  0.0021985 ]
('s:p', 801)
19
('Misclassidied: ', 19)
('Accuracy: ', 97.0)

498000) Percent completed: 99
Error:[ 0.0007265   0.00038389  0.00703498  0.00470844  0.00449681  0.00058547
  0.00